In [29]:
from heapq import merge
from operator import index
from sys import displayhook
import mysql.connector
from mysql.connector import Error
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector as sql
import pandas as pd
from IPython.display import display

# 1. Pilih Database
connection = mysql.connector.connect(host='localhost',
                                         database='cargo_route', # masukan nama database di kiri
                                         user='root',
                                         password='')
cursor = connection.cursor(dictionary=True)

In [30]:
#2. masukkan querry 
#a. querry untuk table sample (sample rute dengan asal dan tujuan berupa int/kode)
samplequery = "SELECT * FROM sample;" 
masterquery = "SELECT * FROM city;"

In [31]:
#3. map sebelum di scale 
maps = pd.read_sql(samplequery,connection)
route = pd.read_sql(samplequery,connection)
display(maps)

C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,sample_id,origin,destination,distance,price,rate,duration,airline
0,1,1,3,430,13000,3.5,2,
1,2,1,5,537,15500,4.0,3,
2,3,1,4,407,12500,4.0,2,
3,4,1,10,909,19000,4.0,7,
4,5,1,11,1259,20250,4.0,10,
5,6,1,9,917,19250,4.5,7,
6,7,1,14,1397,21000,4.5,11,
7,8,1,7,671,16500,4.0,4,
8,9,1,6,808,18250,4.5,6,
9,10,3,11,1025,19750,4.5,8,


In [34]:
from sklearn.preprocessing import MinMaxScaler

norm  = maps[maps.columns[3:7]]
scaler = MinMaxScaler(feature_range=(5, 10))
scaler.fit(norm)
normed=scaler.transform(norm)

maps[maps.columns[3:7]] = normed
maps

,sample_id,origin,destination,distance,price,rate,duration,airline
0,1,1,3,5.610445,6.250000,6.25,5.454545,
1,2,1,5,5.988003,7.291667,7.50,5.909091,
2,3,1,4,5.529287,6.041667,7.50,5.454545,
3,4,1,10,7.300635,8.750000,7.50,7.727273,
4,5,1,11,8.535639,9.270833,7.50,9.090909,
5,6,1,9,7.328864,8.854167,8.75,7.727273,
6,7,1,14,9.022583,9.583333,8.75,9.545455,
7,8,1,7,6.460833,7.708333,7.50,6.363636,
8,9,1,6,6.944248,8.437500,8.75,7.272727,
9,10,3,11,7.709951,9.062500,8.75,8.181818,


In [35]:
from sklearn.preprocessing import MinMaxScaler

norm  = route[route.columns[3:7]]
scaler = MinMaxScaler(feature_range=(5, 10))
scaler.fit(norm)
normed=scaler.transform(norm)

route[route.columns[3:7]] = normed
route

,sample_id,origin,destination,distance,price,rate,duration,airline
0,1,Jakarta,Yogyakarta,5.610445,6.250000,6.25,5.454545,
1,2,Jakarta,Solo,5.988003,7.291667,7.50,5.909091,
2,3,Jakarta,Semarang,5.529287,6.041667,7.50,5.454545,
3,4,Jakarta,Palangkaraya,7.300635,8.750000,7.50,7.727273,
4,5,Jakarta,Balikpapan,8.535639,9.270833,7.50,9.090909,
5,6,Jakarta,Banjarmasin,7.328864,8.854167,8.75,7.727273,
6,7,Jakarta,Makassar,9.022583,9.583333,8.75,9.545455,
7,8,Jakarta,Malang,6.460833,7.708333,7.50,6.363636,
8,9,Jakarta,Surabaya,6.944248,8.437500,8.75,7.272727,
9,10,Yogyakarta,Balikpapan,7.709951,9.062500,8.75,8.181818,


In [36]:
city = pd.read_sql(masterquery,connection)
display(city)

C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,city_id,city_name
0,1,Jakarta
1,2,Bandung
2,3,Yogyakarta
3,4,Semarang
4,5,Solo
5,6,Surabaya
6,7,Malang
7,8,Denpasar
8,9,Banjarmasin
9,10,Palangkaraya


In [37]:
#map dengan asal dan tujuan string bukan int (hanya untuk cek/tampilkan daftar rute)
def map_transformed(origin,destination):
    for kodeC in maps[maps.columns[1]].values: # ambil kode primary yang ada di kolom origin
        for k,v in city.values: # ambil PK dan nama kota yang ada di table kota
            if kodeC == k: # jika PK di kolom origin yang sedang di baca sama dengan salah satu Pk di table kota,
                origin.append(v) # tambahkan nama kotanya.
    route[route.columns[1]] = origin # ubah kolom origin di dataframe berdasarkan list penampung
# untuk destination kurang lebih sama
    for kodeC in maps[maps.columns[2]].values:
        for k,v in city.values:
            if kodeC == k:
                destination.append(v)
    route[route.columns[2]] = destination
    display(route)
origin = []  #list menampung transformasi primary code di kolom origin
destination = [] #list menampung transformasi primary code di kolom destination
map_transformed(origin,destination)

,sample_id,origin,destination,distance,price,rate,duration,airline
0,1,Jakarta,Yogyakarta,5.610445,6.250000,6.25,5.454545,
1,2,Jakarta,Solo,5.988003,7.291667,7.50,5.909091,
2,3,Jakarta,Semarang,5.529287,6.041667,7.50,5.454545,
3,4,Jakarta,Palangkaraya,7.300635,8.750000,7.50,7.727273,
4,5,Jakarta,Balikpapan,8.535639,9.270833,7.50,9.090909,
5,6,Jakarta,Banjarmasin,7.328864,8.854167,8.75,7.727273,
6,7,Jakarta,Makassar,9.022583,9.583333,8.75,9.545455,
7,8,Jakarta,Malang,6.460833,7.708333,7.50,6.363636,
8,9,Jakarta,Surabaya,6.944248,8.437500,8.75,7.272727,
9,10,Yogyakarta,Balikpapan,7.709951,9.062500,8.75,8.181818,


In [38]:
def graph_maker(graph):
    w1,w2,w3,w4 = (1,1,1,1)
    for row in maps.values:
        origin_db = row[1]
        destination_db = row[2]
        distance_db = row[3]
        price_db = row[4]
        rating_db = row[5]
        duration_db = row[6]
        
        W = (distance_db*1/w1) + (price_db*1/w2) + (-rating_db*1/w3) + (duration_db*1/w4)

        if origin_db in graph:
            graph[origin_db][destination_db] = W
        else:
            graph[origin_db]={}
            graph[origin_db][destination_db] = W
    return print(graph)
graph = {}
graph_maker(graph)

{1: {3: 11.064990055815745, 5: 11.688760398622783, 4: 9.525499347747056, 10: 16.27790787194457, 11: 19.397381097495774, 9: 15.1603031906931, 14: 19.40137080045337, 7: 13.032802442206112, 6: 13.904475684865593}, 3: {11: 16.204268781677037}, 4: {6: 8.75}, 6: {2: 13.23284895532602, 8: 10.787314749470713, 14: 14.535777677979514, 17: 20.0, 9: 11.672200124034987}, 9: {11: 11.122677017172428}, 11: {12: 12.659855435084793, 13: 10.139766578986762, 14: 11.402798272064327}, 13: {14: 12.639766578986762}, 14: {18: 16.662670280789545, 15: 11.200913956074507, 16: 12.775677113406472, 17: 16.2469258570176}}


In [40]:
def graph_transformed(route_graph):
    w1,w2,w3,w4 = (1,1,1,1)
    for row in route.values:
        origin_db = row[1]
        destination_db = row[2]
        distance_db = row[3]
        distance_db = row[3]
        price_db = row[4]
        rating_db = row[5]
        duration_db = row[6]
        
        W = (distance_db*1/w1) + (price_db*1/w2) + (-rating_db*1/w3) + (duration_db*1/w4)

        if origin_db in route_graph:
            route_graph[origin_db][destination_db] = W
        else:
            route_graph[origin_db]={}
            route_graph[origin_db][destination_db] = W
    return print(route_graph)
transformed_graph = {}
graph_transformed(transformed_graph)

{'Jakarta': {'Yogyakarta': 11.064990055815745, 'Solo': 11.688760398622783, 'Semarang': 9.525499347747056, 'Palangkaraya': 16.27790787194457, 'Balikpapan': 19.397381097495774, 'Banjarmasin': 15.1603031906931, 'Makassar': 19.40137080045337, 'Malang': 13.032802442206112, 'Surabaya': 13.904475684865593}, 'Yogyakarta': {'Balikpapan': 16.204268781677037}, 'Semarang': {'Surabaya': 8.75}, 'Surabaya': {'Bandung': 13.23284895532602, 'Denpasar': 10.787314749470713, 'Makassar': 14.535777677979514, 'Manado': 20.0, 'Banjarmasin': 11.672200124034987}, 'Banjarmasin': {'Balikpapan': 11.122677017172428}, 'Balikpapan': {'Tarakan': 12.659855435084793, 'Palu': 10.139766578986762, 'Makassar': 11.402798272064327}, 'Palu': {'Makassar': 12.639766578986762}, 'Makassar': {'Ambon': 16.662670280789545, 'Kendari': 11.200913956074507, 'Gorontalo': 12.775677113406472, 'Manado': 16.2469258570176}}


In [41]:
def Unpacking_Track(Close_list):    
    List_lenght = len(Close_list)
    Result = []
    Child_list = {}
    Result.append(Close_list[-1])
    for l in range (List_lenght):
        Current_node = Close_list[-1]
        print("Current Node : ",Current_node)
        print(Close_list)
        if Current_node in maps:
            Child_list=(maps[Current_node])
            print("Current Child List : ",Child_list)
            if Result[-1] in Child_list:  
                Result.append(Current_node)
                Close_list.remove(Current_node)
                print('Result :',Result)   

            else:
                Close_list.remove(Current_node)
                print('Result :',Result) 

        else:
            Close_list.remove(Current_node)
            print('Result :',Result) 
            
#     Actual_path = [] 
 
#     # Substract one from the length of the original array to start from the last index: 
#     reducer = len(Result)-1 

#     # Reverse the list inside a for loop: 
#     for i in Result:
#      Actual_path.append(Result[reducer]) # Append the result to the empty list 
#      reducer -=1 # Decrease the index by one at each iteration using the reducer
    
    return Result

In [42]:
def Sorting_Path(Result):
    Actual_path = [] 
 
    # Substract one from the length of the original array to start from the last index: 
    reducer = len(Result)-1 

    # Reverse the list inside a for loop: 
    for i in Result:
     Actual_path.append(Result[reducer]) # Append the result to the empty list 
     reducer -=1 # Decrease the index by one at each iteration using the reducer
    return Actual_path


In [43]:
#Parameters
origin = 'Jakarta'
destination = 'Palu'
maps = transformed_graph

#Pivate Variables
OPEN = {origin:0}
print("LABEL :",OPEN)
print("")

N = []

CLOSE = []

while destination not in CLOSE:
    print("MAPS :",maps)
    print("")
    print("OPEN LIST:",OPEN)
    print("")
    print("CLOSE LIST:",CLOSE)
    print("")
    if OPEN == {}:
       print("No Route !")
       break
    else:
        bestNode = min(OPEN,key=OPEN.get)
        childNode = {}
        print("BEST NODES:",bestNode)
        print("")
        
        if bestNode in CLOSE:
            del OPEN[bestNode]
        else:
            if bestNode in maps:
                childNode.update(maps[bestNode])
                print("BESTNODE'S CHILD :",childNode)
                print("")

                #pelabelan Open List
                for node,item in childNode.items():
                    print("NODE",node)
                    print("d(u) =",OPEN[bestNode])
                    print("d(v)=",OPEN[bestNode],"+",item)
                    newChild = {}
                    newChild[node] = OPEN[bestNode]+item
                    print(newChild)
                    print("")

                    if node in OPEN:
                        if newChild[node] < OPEN[node]:
                            print("BOBOT LAMA :",OPEN[node])
                            print("BOBOT BARU :",newChild[node])
                            print("")
                            OPEN.update(newChild)


                    else:
                        OPEN.update(newChild) 

                del OPEN[bestNode]
                CLOSE.append(bestNode)

            else:
                del OPEN[bestNode]
            
print("Open List Terakhir :",OPEN)
print("Close List Terakhir :",CLOSE)
        
        




LABEL : {'Jakarta': 0}

MAPS : {'Jakarta': {'Yogyakarta': 11.064990055815745, 'Solo': 11.688760398622783, 'Semarang': 9.525499347747056, 'Palangkaraya': 16.27790787194457, 'Balikpapan': 19.397381097495774, 'Banjarmasin': 15.1603031906931, 'Makassar': 19.40137080045337, 'Malang': 13.032802442206112, 'Surabaya': 13.904475684865593}, 'Yogyakarta': {'Balikpapan': 16.204268781677037}, 'Semarang': {'Surabaya': 8.75}, 'Surabaya': {'Bandung': 13.23284895532602, 'Denpasar': 10.787314749470713, 'Makassar': 14.535777677979514, 'Manado': 20.0, 'Banjarmasin': 11.672200124034987}, 'Banjarmasin': {'Balikpapan': 11.122677017172428}, 'Balikpapan': {'Tarakan': 12.659855435084793, 'Palu': 10.139766578986762, 'Makassar': 11.402798272064327}, 'Palu': {'Makassar': 12.639766578986762}, 'Makassar': {'Ambon': 16.662670280789545, 'Kendari': 11.200913956074507, 'Gorontalo': 12.775677113406472, 'Manado': 16.2469258570176}}

OPEN LIST: {'Jakarta': 0}

CLOSE LIST: []

BEST NODES: Jakarta

BESTNODE'S CHILD : {'Yogya

In [44]:
a=Unpacking_Track(CLOSE)
b=Sorting_Path(a)

print(b)

Current Node :  Palu
['Jakarta', 'Semarang', 'Yogyakarta', 'Surabaya', 'Banjarmasin', 'Balikpapan', 'Makassar', 'Palu']
Current Child List :  {'Makassar': 12.639766578986762}
Result : ['Palu']
Current Node :  Makassar
['Jakarta', 'Semarang', 'Yogyakarta', 'Surabaya', 'Banjarmasin', 'Balikpapan', 'Makassar']
Current Child List :  {'Ambon': 16.662670280789545, 'Kendari': 11.200913956074507, 'Gorontalo': 12.775677113406472, 'Manado': 16.2469258570176}
Result : ['Palu']
Current Node :  Balikpapan
['Jakarta', 'Semarang', 'Yogyakarta', 'Surabaya', 'Banjarmasin', 'Balikpapan']
Current Child List :  {'Tarakan': 12.659855435084793, 'Palu': 10.139766578986762, 'Makassar': 11.402798272064327}
Result : ['Palu', 'Balikpapan']
Current Node :  Banjarmasin
['Jakarta', 'Semarang', 'Yogyakarta', 'Surabaya', 'Banjarmasin']
Current Child List :  {'Balikpapan': 11.122677017172428}
Result : ['Palu', 'Balikpapan', 'Banjarmasin']
Current Node :  Surabaya
['Jakarta', 'Semarang', 'Yogyakarta', 'Surabaya']
Curre